# Linkres cleanup and filtering

### Boilerplate

In [1]:
%load_ext lab_black
# python internal
import collections
import copy
import gc
from glob import glob
import h5py
import itertools
import os
import random
import re
import socket
import shutil
import subprocess
import sys

# conda/pip
import dask
import graphviz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
from tqdm import tqdm

# special packages on the DIGS
import py3Dmol
import pymol
import pyrosetta

# notebook magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

print(os.getcwd())
print(socket.gethostname())

/mnt/home/pleung/projects/bistable_bundle/r4/hinges
dig96


### Generate state X and state Y with and without disulfides by doing `KeepSequenceSymmetry` and fixbb design
I will use the serialization build of PyRosetta to enable recording user defined info about the designs.  
This enables downstream inline filtering and data analysis, as well as clustering by lineage.

Record rich information about residues that might be interesting to show/direct switching, GLY, HIS and TRP.  
Record info about sequences and save pdbs for X and Y, with and without disulfide staples  

In [2]:
from pyrosetta.distributed.packed_pose.core import PackedPose


def almost_linkres(packed_pose_in=None, **kwargs) -> PackedPose:
    """
    Load pose. Design with linkres x3: state_X_nocys, state_Y_nocys,
    state_XY_nocys. Dump X and Y poses with and without disulfides (4 total).
    Return XY without the disulfides.
    """
    import os
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed import cluster
    from pyrosetta.rosetta.core.pack.task.operation import (
        IncludeCurrent,
        InitializeFromCommandline,
        ExtraRotamersGeneric,
    )
    from pyrosetta.rosetta.protocols.denovo_design.movers import FastDesign
    from pyrosetta.rosetta.protocols.rosetta_scripts import XmlObjects
    from pyrosetta.rosetta.protocols.task_operations import LimitAromaChi2Operation

#     def parse_scoreinfo(pdb_path) -> dict:
#         """
#         Hacky unsafe way to load scoreinfo written to pdb by rosetta.
#         Probably should add additional pdb strings to skip
#         """
#         with open(pdb_path, "r") as f:
#             scores = {}
#             pdb_strings = ["HEADER", "EXPDTA", "SSBOND", "ATOM", "TER", "REMARK", "#"]
#             for line in f:
#                 if line == "\n":  # skip empty lines
#                     pass
#                 # skip pdb specific rows
#                 elif any(pdb_string in line for pdb_string in pdb_strings):
#                     pass
#                 else:
#                     key_value = line.rstrip().split()
#                     # handle pose energies table
#                     try:
#                         assert len(key_value) == 2
#                     except AssertionError:  # pose energies have many values per (residue) key
#                         continue
#                     key, value = tuple(key_value)
#                     if any(c.isalpha() for c in value):  # handle as str below
#                         pass
#                     elif "," in value:  # handle as str below
#                         pass
#                     elif "." in value:  # if it has a . it maybe a float
#                         try:
#                             value = float(value)
#                         except ValueError:
#                             pass
#                     else:  # if all else fails maybe an int
#                         try:
#                             value = int(value)
#                         except ValueError:
#                             pass
#                     scores[key] = value  # if all typecasting failed it is still a str
#         return scores

    def score(packed_pose_in: PackedPose, **kwargs) -> PackedPose:
        """
        Sequence
        Compute HIS SASA for X, Y, (compute delta on df)
        Compute buried HIS (probably in interface)
        Count GLY near interface, (compute glycine count on df)
        Compute TRP SASA for X, Y, (compute delta on df)
        Count TRP nearby aromatic non-TRP neighbors
        Assumes middle is where the switch is split originally, enabling forward and backward SSElement selection approach
        @pleung
        """
        import pyrosetta
        import pyrosetta.distributed.io as io
        from pyrosetta.distributed.tasks.rosetta_scripts import (
            SingleoutputRosettaScriptsTask,
        )

        scores = kwargs["score_dict"]
        xml = """
        <ROSETTASCRIPTS>
            <SCOREFXNS>
            </SCOREFXNS>
            <RESIDUE_SELECTORS>
                <ResidueName name="GLY" residue_name3="GLY" />
                <ResidueName name="HIS" residue_name3="HIS" />
                <ResidueName name="TRP" residue_name3="TRP" />
                <ResidueName name="aro" residue_name3="TYR,PHE,HIS" />
                <Layer name="surface" select_core="false" select_boundary="false" select_surface="true"
                    use_sidechain_neighbors="true"/>
                <Layer name="boundary" select_core="false" select_boundary="true" select_surface="false" 
                    use_sidechain_neighbors="true"/>
                <Layer name="core" select_core="true" select_boundary="false" select_surface="false" 
                    use_sidechain_neighbors="true"/>
                <SecondaryStructure name="sheet" overlap="0" minH="3" minE="2" include_terminal_loops="false" 
                    use_dssp="true" ss="E"/>
                <SecondaryStructure name="entire_loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" 
                    use_dssp="true" ss="L"/>
                <SecondaryStructure name="entire_helix" overlap="0" minH="3" minE="2" include_terminal_loops="false"
                    use_dssp="true" ss="H"/>
                <And name="helix_cap" selectors="entire_loop">
                    <PrimarySequenceNeighborhood lower="1" upper="0" selector="entire_helix"/>
                </And>
                <And name="helix_start" selectors="entire_helix">
                    <PrimarySequenceNeighborhood lower="0" upper="1" selector="helix_cap"/>
                </And>
                <And name="helix" selectors="entire_helix">
                    <Not selector="helix_start"/>
                </And>
                <And name="loop" selectors="entire_loop">
                    <Not selector="helix_cap"/>
                </And>
                <SSElement name="near_int" selection="{pre},H,S" to_selection="-{post},H,E" chain="A" reassign_short_terminal_loop="2" />            
                <Neighborhood name="near_TRP" selector="TRP" distance="6.0" />
                <And name="GLY_near_int" selectors="near_int,HIS" />
                <And name="core_HIS" selectors="core,HIS" />
                <And name="aro_near_TRP" selectors="aro,near_TRP" />
            </RESIDUE_SELECTORS>
            <TASKOPERATIONS>
            </TASKOPERATIONS>
            <FILTERS>
                <BuriedSurfaceArea name="HIS_burial" residue_selector="HIS" confidence="0" />
                <BuriedSurfaceArea name="TRP_burial" residue_selector="TRP" confidence="0" />
                <ResidueCount name="buried_HIS" residue_selector="core_HIS" />
                <ResidueCount name="near_int_GLY" residue_selector="GLY_near_int" />
                <ResidueCount name="near_TRP_aro" residue_selector="aro_near_TRP" />
            </FILTERS>
            <MOVERS>
            </MOVERS>
            <PROTOCOLS>
                <Add filter="HIS_burial" />
                <Add filter="TRP_burial" />
                <Add filter="buried_HIS" />
                <Add filter="near_int_GLY" />
                <Add filter="near_TRP_aro" />
            </PROTOCOLS>
        </ROSETTASCRIPTS>
        """.format(
            pre=int(int(scores["pre_break_helix"]) - 1),
            post=int(int(scores["pre_break_helix"]) - 1),
        )
        # relax and score pose
        score = SingleoutputRosettaScriptsTask(xml)
        scored_ppose = score(packed_pose_in.pose.clone())
        update = scored_ppose.pose.scores
        pose = io.to_pose(scored_ppose.pose.clone())
        # update scores for values that were updated
        for key, value in scores.items():
            if key in update.keys():
                pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, key, update[key])
            else:
                pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, key, value)
        final_ppose = io.to_packed(pose)
        return final_ppose

    if packed_pose_in == None:
        file = kwargs["-s"]
        with open(file, "rb") as f:
            ppose = io.pose_from_pdbstring(bz2.decompress(f.read()).decode())
        scores = pyrosetta.distributed.cluster.get_scores_dict(file)["scores"]
    else:
        raise RuntimeError("Need to supply an input")
    
    pose = io.to_pose(ppose)
    
    if kwargs["out_path"] == None:
        out_path = os.getcwd()
    else:
        out_path = kwargs["out_path"]
        os.makedirs(os.path.join(os.getcwd(), out_path), exist_ok=True)
    cys_X = scores["disulfide_at_X"].split(",")
    cys_Y = scores["disulfide_at_Y"].split(",")
    unique_cys = set(cys_X + cys_Y)
    # check whether disulfides are orthogonal
    design_resis = [int(x) for x in unique_cys]
    try:
        assert len(unique_cys) in [
            3,
            4,
        ]  # if there are less than 3 CYS or more than 4 we really f'd up at some point
    except AssertionError:
        print("There should be at least 3 unique CYS, but no more than 4")
        return None
    cys1_X, cys2_X = int(cys_X[0]), int(cys_X[1])
    cys1_Y, cys2_Y = int(cys_Y[0]), int(cys_Y[1])
    length = scores["total_length"]
    resis_matrix = [[x, int(x + length)] for x in design_resis]
    target_resis = ",".join(
        [str(x) for row in resis_matrix for x in row]
    )  # flatten matrix
    xml_string = """
    <ROSETTASCRIPTS>
        <SCOREFXNS>
            <ScoreFunction name="sfxn_design" weights="beta_nov16" >
                <Set use_hb_env_dep="true" />
                <Reweight scoretype="approximate_buried_unsat_penalty" weight="17" />
                <Set approximate_buried_unsat_penalty_burial_atomic_depth="3.5" />
                <Set approximate_buried_unsat_penalty_hbond_energy_threshold="-1.0" />
                <Set approximate_buried_unsat_penalty_natural_corrections1="true" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_cross_chain="-7" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_ser_to_helix_bb="1"/>                    
            </ScoreFunction>
        </SCOREFXNS>
        <RESIDUE_SELECTORS>
            <Chain name="chA" chains="A" />
            <Chain name="chB" chains="B" />
            <Index name="designable" resnums="{target_resis}" />
            <Neighborhood name="packable" selector="designable" distance="6.0" include_focus_in_subset="true" />
            <Not name="not_designable" selector="designable" />
            <Not name="not_packable" selector="packable" />
        </RESIDUE_SELECTORS>
        <TASKOPERATIONS>
            <OperateOnResidueSubset name="design" selector="designable"> # no CYS; GLY; PRO;
                <RestrictAbsentCanonicalAASRLT aas="ADEFHIKLMNQRSTVWY" />
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="pack" selector="not_designable">
                <RestrictToRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="lock" selector="not_packable">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>
            <RestrictToRepacking name="rtrp" />
            <KeepSequenceSymmetry name="2state" setting="true"/>
        </TASKOPERATIONS>
        <MOVERS>
            <SetupForSequenceSymmetryMover name="almost_linkres" sequence_symmetry_behaviour="2state" >
                <SequenceSymmetry residue_selectors="chA,chB" />
            </SetupForSequenceSymmetryMover>
            <SwitchChainOrder name="delete_Y" chain_order="12"/>
            <SwitchChainOrder name="delete_X" chain_order="34"/>
            <MutateResidue name="cys1_X" target="{cys1_X}" new_res="CYS" />
            <MutateResidue name="cys2_X" target="{cys2_X}" new_res="CYS" />
            <MutateResidue name="cys1_Y" target="{cys1_Y}" new_res="CYS" />
            <MutateResidue name="cys2_Y" target="{cys2_Y}" new_res="CYS" />
            <ForceDisulfides name="restore_X" 
                scorefxn="sfxn_design"
                disulfides="{cys1_X}:{cys2_X}"
                remove_existing="false"
                repack="true" />
            <ForceDisulfides name="restore_Y" 
                scorefxn="sfxn_design"
                disulfides="{cys1_Y}:{cys2_Y}"
                remove_existing="false"
                repack="true" />
            <FastRelax name="relax" 
                scorefxn="sfxn_design"
                repeats="1" 
                relaxscript="MonomerRelax2019"
                />
            <PackRotamersMover name="repack" 
                scorefxn="sfxn_design"
                task_operations="rtrp"
                />
        </MOVERS>
        <FILTERS>
        </FILTERS>
        <SIMPLE_METRICS>
        </SIMPLE_METRICS>
        <APPLY_TO_POSE>
        </APPLY_TO_POSE>
        <PROTOCOLS>
        </PROTOCOLS>
    </ROSETTASCRIPTS>
    """.format(
        target_resis=target_resis,
        cys1_X=cys1_X,
        cys2_X=cys2_X,
        cys1_Y=cys1_Y,
        cys2_Y=cys2_Y,
    )
    # set taskops by extracting them from the xml_string
    xml_obj = XmlObjects.create_from_string(xml_string)
    design_task = xml_obj.get_task_operation("design")
    pack_task = xml_obj.get_task_operation("pack")
    lock_task = xml_obj.get_task_operation("lock")
    linkres_task = xml_obj.get_task_operation("2state")
    # set taskops
    task_factory = pyrosetta.rosetta.core.pack.task.TaskFactory()
    task_factory.push_back(IncludeCurrent())
    task_factory.push_back(InitializeFromCommandline())
    arochi = LimitAromaChi2Operation()
    arochi.include_trp(True)
    task_factory.push_back(arochi)
    ex1_ex2 = ExtraRotamersGeneric()
    ex1_ex2.ex1(True)
    ex1_ex2.ex2(True)
    task_factory.push_back(ex1_ex2)
    task_factory.push_back(design_task)
    task_factory.push_back(pack_task)
    task_factory.push_back(lock_task)
    task_factory.push_back(linkres_task)
    # set movemap
    mm = pyrosetta.rosetta.core.kinematics.MoveMap()
    mm.set_bb(False)
    mm.set_chi(True)
    mm.set_jump(False)
    sfxn_design = xml_obj.get_score_function("sfxn_design")
    fast_design = FastDesign(scorefxn_in=sfxn_design, standard_repeats=1)
    fast_design.cartesian(False)
    fast_design.set_task_factory(task_factory)
    fast_design.set_movemap(mm)
    fast_design.minimize_bond_angles(False)
    fast_design.minimize_bond_lengths(False)
    fast_design.min_type("lbfgs_armijo_nonmonotone")
    fast_design.ramp_down_constraints(False)

    name = pose.pdb_info().name()
    basename_no_ext = name.split("/")[-1].replace(".pdb", "", 1)
    if out_path[-1] != "/":
        out_path += "/"
    else:
        pass
    pose = pose.clone()
    almost_linkres = xml_obj.get_mover("almost_linkres")
    almost_linkres.apply(pose)
    fast_design.apply(pose)

    ref_pose = pose.clone()
    # get state X
    delete_Y = xml_obj.get_mover("delete_Y")
    delete_Y.apply(pose)
    ref_X = pose.clone()
    cys1 = xml_obj.get_mover("cys1_X")
    cys2 = xml_obj.get_mover("cys2_X")
    restore = xml_obj.get_mover("restore_X")
    relax = xml_obj.get_mover("relax")
    repack = xml_obj.get_mover("repack")
    cys1.apply(pose)
    cys2.apply(pose)
    restore.apply(pose)
    repack.apply(pose)
    # dump X
    pose.dump_pdb(os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_X.pdb"))
    scores["X_seq"] = pose.sequence()
    # get rmsd
    relax_X = ref_X.clone()
    relax.apply(relax_X)
    # dump X_nocys
    relax_X.dump_pdb(
        os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_X_nocys.pdb")
    )
    rmsd_calc = pyrosetta.rosetta.core.simple_metrics.metrics.RMSDMetric()
    rmsd_calc.set_rmsd_type(pyrosetta.rosetta.core.scoring.rmsd_atoms(3))
    rmsd_calc.set_run_superimpose(True)
    rmsd_calc.set_comparison_pose(ref_X)
    rmsd = rmsd_calc.calculate(relax_X)
    scores["rmsd_final_X"] = rmsd
    # get state Y
    pose = ref_pose.clone()
    delete_X = xml_obj.get_mover("delete_X")
    delete_X.apply(pose)
    ref_Y = pose.clone()
    cys1 = xml_obj.get_mover("cys1_Y")
    cys2 = xml_obj.get_mover("cys2_Y")
    restore = xml_obj.get_mover("restore_Y")
    cys1.apply(pose)
    cys2.apply(pose)
    restore.apply(pose)
    repack.apply(pose)
    # dump Y
    pose.dump_pdb(os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_Y.pdb"))
    scores["Y_seq"] = pose.sequence()
    # get rmsd
    relax_Y = ref_Y.clone()
    relax.apply(relax_Y)
    # dump Y_nocys
    relax_Y.dump_pdb(
        os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_Y_nocys.pdb")
    )
    rmsd_calc = pyrosetta.rosetta.core.simple_metrics.metrics.RMSDMetric()
    rmsd_calc.set_rmsd_type(pyrosetta.rosetta.core.scoring.rmsd_atoms(3))
    rmsd_calc.set_run_superimpose(True)
    rmsd_calc.set_comparison_pose(ref_Y)
    rmsd = rmsd_calc.calculate(relax_Y)
    scores["rmsd_final_Y"] = rmsd
    assert relax_X.sequence() == relax_Y.sequence()
    scores["final_sequence"] = relax_X.sequence()
    # put in paths to X, Y, X_nocys, Y_nocys
    for suffix in ["X", "Y", "X_nocys", "Y_nocys"]:
        scores[f"{suffix}_path"] = os.path.join(
            os.getcwd(), out_path + f"{basename_no_ext}_{suffix}.pdb"
        )
    # score relax_X and relax_Y, update scores with those scores
    scored_X = score(io.to_packed(relax_X), score_dict=scores)
    scored_Y = score(io.to_packed(relax_Y), score_dict=scores)
    updates = []
    keys = ["HIS_burial", "TRP_burial", "buried_HIS", "near_int_GLY", "near_TRP_aro"]

    for chain, score_dict in [
        ("X", dict(scored_X.pose.scores)),
        ("Y", dict(scored_Y.pose.scores)),
    ]:
        updates.append(
            {
                f"{key}_{chain}": value
                for key, value in score_dict.items()
                if key in keys
            }
        )
    scores.update({**updates[0], **updates[1]})
    for key, value in scores.items():
        pyrosetta.rosetta.core.pose.setPoseExtraScore(ref_pose, key, value)
    ppose = io.to_packed(ref_pose)
    return ppose

### Setup dask, set command line options, make tasks and submit to client for cleanup
Not using:  
`-indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5`  
`-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc`

In [3]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
import logging
import pwd
from pyrosetta.distributed.cluster.core import PyRosettaCluster


logging.basicConfig(level=logging.INFO)
selected = os.path.join(os.getcwd(), "05_filter/1trp_good.list")
options = {
    "-out:level": "300",
    "-detect_disulf": "false",
    "-precompute_ig": "true",
}


def create_tasks(selected, options):
    with open(selected, "r") as f:
        for i, pdb_path in enumerate(f):
            tasks = {"options": "-corrections::beta_nov16 true"}
            tasks["extra_options"] = options
            tasks["-s"] = pdb_path.rstrip()
            tasks["out_path"] = f"06_states/{int(i / 250)}"
            yield tasks


print("run the following from your local terminal:")
print(
    f"ssh -L 8000:localhost:8787 {pwd.getpwuid(os.getuid()).pw_name}@{socket.gethostname()}"
)

output_path = os.path.join(os.getcwd(), "06_cleanup")

if __name__ == "__main__":
    # configure SLURM cluster as a context manager
    with SLURMCluster(
        cores=1,
        processes=1,
        job_cpu=1,
        memory="4GB",
        queue="medium",
        walltime="23:30:00",
        death_timeout=120,
        local_directory="$TMPDIR/dask",
        log_directory="/mnt/home/pleung/logs/slurm_logs",
        extra=["--lifetime", "23h", "--lifetime-stagger", "4m"],
    ) as cluster:
        print(cluster.job_script())
        # scale between 1-1020 workers,
        cluster.adapt(
            minimum=1,
            maximum=1020,
            wait_count=400,  # Number of consecutive times that a worker should be suggested for removal it is removed
            interval="5s",  # Time between checks
        )
        # setup a client to interact with the cluster as a context manager
        with Client(cluster) as client:
            print(client)
            PyRosettaCluster(
                tasks=create_tasks(selected, options),
                client=client,
                scratch_dir=output_path,
                output_path=output_path,
            ).distribute(protocols=[almost_linkres])

run the following from your local terminal:
ssh -L 8000:localhost:8787 pleung@dig96
#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /mnt/home/pleung/logs/slurm_logs/dask-worker-%J.err
#SBATCH -o /mnt/home/pleung/logs/slurm_logs/dask-worker-%J.out
#SBATCH -p medium
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH -t 23:30:00

JOB_ID=${SLURM_JOB_ID%;*}

/home/pleung/.conda/envs/phil/bin/python -m distributed.cli.dask_worker tcp://172.16.131.126:44353 --nthreads 1 --memory-limit 3.73GiB --name name --nanny --death-timeout 120 --local-directory $TMPDIR/dask --lifetime 23h --lifetime-stagger 4m

<Client: 'tcp://172.16.131.126:44353' processes=0 threads=0, memory=0 B>


INFO:pyrosetta.distributed:maybe_init performing pyrosetta initialization: {'options': '-run:constant_seed 1 -multithreading:total_threads 1', 'extra_options': '-mute all', 'set_logging_handler': 'interactive', 'silent': True}
INFO:pyrosetta.rosetta:Found rosetta database at: /home/pleung/.conda/envs/phil/lib/python3.8/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2021 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release 2021.12+release.ed6a5560506cfd327d4a6a3e2c9b0c9f6f4a6535 2021-03-26T16:09:25] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [ ]:
%%time
import pyrosetta
from pyrosetta.distributed import cluster
import pyrosetta.distributed.io as io

flags = """
-out:level 300
-precompute_ig 1
-detect_disulf false
-corrections::beta_nov16 true
-holes:dalphaball /home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc
-indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5
"""
pyrosetta.distributed.init(" ".join(flags.replace("\n\t", " ").split()))
t = almost_linkres(
    None,
    **{
        "-s": "/mnt/home/pleung/projects/bistable_bundle/r4/hinges/05_msd_runs_1/10/0009d2892b7dbae457c1c6f8bc40fe5524115bd1423ebeb4.pdb",
        "out_path": "test",
    }
)